In [8]:
# sklearn necesita una version de numpy anterior a numpy 2
!pip install "numpy<2"

import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [6]:
# función para cargar las imagenes de entrenamiento
def cargar_datos(data_dir):
    X = []
    y = []
    for class_name in ['margaritas', 'lirios', 'rosas']:
        class_dir = os.path.join(data_dir, class_name)
        for file_name in os.listdir(class_dir):
            file_path = os.path.join(class_dir, file_name)
            image = Image.open(file_path).convert('L')  # Convert to grayscale
            image = image.resize((64, 64))  # Resize to a fixed size
            image = np.array(image).flatten()  # Flatten the image
            X.append(image)
            y.append(class_name)
    return np.array(X), np.array(y)

# Carga los datos, x tiene los datos y y tiene las etiquetas (los )
data_dir = 'imagenes'
x, y = cargar_datos(data_dir)

In [7]:
# Codifica las etiquetas
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Dicide los datos entre los de entrenamiento y los de prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)